# Домашнее задание - линейная регрессия

### Работа с признаками (8 баллов)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv 


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж 
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные функции onehot_encode() и minmax_scale().

**Подсказка**: маску для категориальных признаков можно сделать фильтром cat_features_mask = (df.dtypes == "object").values

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


def onehot_encode(data, cat_features_mask):
    # Создание пустого DataFrame для закодированных признаков
    encoded_data = pd.DataFrame()
    
    # Закодирование категориальных признаков с помощью one-hot encoding
    for i, is_cat_feature in enumerate(cat_features_mask):
        if is_cat_feature:
            encoded_cols = pd.get_dummies(data.iloc[:, i], prefix=data.columns[i])
            encoded_data = pd.concat([encoded_data, encoded_cols], axis=1)
        else:
            encoded_data = pd.concat([encoded_data, data.iloc[:, i]], axis=1)
    
    return encoded_data


def minmax_scale(data, scaler):
    # Масштабирование данных с помощью MinMaxScaler
    scaled_data = scaler.fit_transform(data)
    
    # Преобразование масштабированных данных в DataFrame
    scaled_data = pd.DataFrame(scaled_data, columns=data.columns)
    
    return scaled_data


# Загрузка данных
url = "https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv"
data = pd.read_csv(url)

# Обработка пропущенных значений
data.fillna(0, inplace=True)

# Создание маски для категориальных признаков
cat_features_mask = (data.dtypes == "object").values

# Применение функции onehot_encode()
data_encoded = onehot_encode(data, cat_features_mask)

# Применение функции minmax_scale()
scaler = MinMaxScaler()
data_scaled = minmax_scale(data_encoded, scaler)
data_scaled.head()









,Agency_ADM,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,Agency_CWT,Agency_EPX,Agency_JWT,Agency_JZI,...,Destination_VIET NAM,"Destination_VIRGIN ISLANDS, U.S.",Destination_ZAMBIA,Destination_ZIMBABWE,Net Sales,Commision (in value),Gender_0,Gender_F,Gender_M,Age
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.300250,0.033757,0.0,1.0,0.0,0.686441
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.300250,0.033757,0.0,1.0,0.0,0.601695
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.283153,0.104762,1.0,0.0,0.0,0.271186
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.291410,0.083810,1.0,0.0,0.0,0.271186
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.307923,0.041905,1.0,0.0,0.0,0.347458


In [7]:
data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,0,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,0,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,0,41


Подробный анализ и подготовка датасета часто помогают улучшить качество модели. Ниже представлено несколько идей преобразований. Вы можете применить одно или несколько из этих преобразований (а можете не применять), чтобы помочь будущей модели. 

1. Посмотрите на количественные признаки. Возможно, в некоторых признаках есть выбросы - значения, которые сильно выбиваются. Такие значения полезно удалять. Советуем присмотреться к колонке Duration)

2. Можно заметить, что one hot encoding сильно раздувает количество столбцов. Радикальное решение - можно попробовать выбросить все категориальные признаки из датасета.

3. Если все-таки оставляете категориальные признаки, то подумайте, как уменьшить количество столбцов после one hot encoding. Признаки с большим количеством значений (Duration - 149! разных стран) можно удалить или попробовать сгруппировать некоторые значения.

4. Downsampling. Датасет достаточно большой, разница в классах огромная. Можно уменьшить число наблюдений с частым ответом.

In [6]:
def downsampling(data, target_column, ratio):
    # Разделение данных на классы
    class_0 = data[data[target_column] == 0]
    class_1 = data[data[target_column] == 1]
    
    # Определение размера выборки для класса 0
    sample_size = int(len(class_0) * ratio)
    
    # Случайное выборка из класса 0
    sampled_class_0 = class_0.sample(sample_size, random_state=42)
    
    # Объединение выборки класса 0 и класса 1
    sampled_data = pd.concat([sampled_class_0, class_1])
    
    return sampled_data


# Уменьшение числа наблюдений с частым ответом
data_downsampled = downsampling(data, 'target_column', 0.2)


KeyError: 'target_column'

### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20. Зафиксируйте random_state.

**Подсказка:** быстро перевести Yes/No в 1/0 можно так - np.where(df['Claim'] == 'Yes', 1,0)

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

# Вынесение признака 'Claim' в вектор ответов
y = np.where(data['Claim'] == 'Yes', 1, 0)

# Удаление признака 'Claim' из датасета
X = data.drop('Claim', axis=1)

# Разделение датасета на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2. 

In [7]:
# посчитайте аналитическое решение

In [ ]:
# посчитать аналитическое решение с регуляризацией

Постройте модель LinearRegression, примените к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)

In [36]:
# обучите модель линейной регрессии LinearRegression на обучающей выборке, примените к тестовой

In [41]:
# посчитайте MSE, предварительно округлив предсказанные ответы до целого

### Вывод (1 балла)

Напишите краткий вывод по заданию (достаточно пары предложений). Расскажите, какие способы предобработки данных вы выбрали и почему. Насколько хороша ваша модель?